<a href="https://colab.research.google.com/github/yagnesh2004/Python/blob/main/TextClassification_Patient_Symptoms_and_Diseases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AIMERS

In [3]:
!pip install transformers
import pandas as pd
import re
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
#from transformers import TfidfVectorizerForTransformers


# Load the data
data = pd.read_csv('symptomssingle.csv')

# Check for any missing values and remove them
data = data.dropna()

# Define a function to separate symptoms and diseases from the text
def separate_symptoms_and_diseases(text):
    symptoms = re.findall(r'{"symptoms":"(.*?)"}', text)
    disease = re.sub(r'(?:{"symptoms":".*?"},?)+', '', text).strip()
    disease = disease.replace('],', '').strip()  # Remove '],' from the disease name
    return symptoms, disease

# Apply the function to the data
data['symptoms_and_diseases'] = data['data'].apply(separate_symptoms_and_diseases)
data[['symptoms', 'disease']] = pd.DataFrame(data['symptoms_and_diseases'].tolist(), index=data.index)
data = data.drop(columns=['data', 'symptoms_and_diseases'])

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Preprocessing function
def preprocess(symptoms):
    processed_symptoms = []
    for symptom in symptoms:
        doc = nlp(symptom)
        processed_symptom = ' '.join(token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha)
        processed_symptoms.append(processed_symptom)
    return ' '.join(processed_symptoms)

# Preprocess the symptoms column
data['symptoms_preprocessed'] = data['symptoms'].apply(preprocess)


# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data['symptoms_preprocessed'], data['disease'], test_size=0.2, random_state=42)


# Create a pipeline for text classification
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('classifier', LogisticRegression(solver='liblinear', C=10))
])
# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Accuracy:  0.0
Classification Report:
                                                 precision    recall  f1-score   support

                          Acanthosis nigricans       0.00      0.00      0.00       0.0
                                     Acariasis       0.00      0.00      0.00       0.0
                                          Acne       0.00      0.00      0.00       0.0
                              Acute bronchitis       0.00      0.00      0.00       1.0
                            Acute bronchospasm       0.00      0.00      0.00       1.0
                                Acute glaucoma       0.00      0.00      0.00       1.0
                            Acute pancreatitis       0.00      0.00      0.00       0.0
                         Acute stress reaction       0.00      0.00      0.00       1.0
                           Adjustment reaction       0.00      0.00  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [4]:
!pip install joblib
import joblib

# Save the entire pipeline, including the TfidfVectorizer and Logistic Regression model
joblib.dump(pipeline, 'DiseasePredictionBasedonSymptoms_Pipeline.joblib')



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


['DiseasePredictionBasedonSymptoms_Pipeline.joblib']

# New Section

In [8]:
import joblib

# Load the saved model
loaded_pipeline = joblib.load('DiseasePredictionBasedonSymptoms_Pipeline.joblib')

# Make predictions using the loaded model (example)
sample_symptom = "headache , vompting , nausia, stomachpain , motions, heavy sweat"
processed_symptom = preprocess([sample_symptom])
prediction = loaded_pipeline.predict([processed_symptom])

print("Predicted disease:", prediction)


Predicted disease: ['Menopause']
